# Tutorial 1.8: Complete RAG Application

## Building a Production-Ready RAG System with Full Observability

Welcome to the final notebook! This brings together everything you've learned to build a complete, production-ready RAG (Retrieval-Augmented Generation) application with comprehensive MLflow tracking and tracing.

### What You'll Build

A full RAG system that includes:
- ✅ Document embedding and indexing
- ✅ Semantic search / retrieval
- ✅ Context-aware response generation
- ✅ Complete experiment tracking
- ✅ End-to-end tracing
- ✅ Cost monitoring
- ✅ Performance metrics
- ✅ Error handling
- ✅ Caching strategies
- ✅ RAG evaluation with RAGAS metrics

### Prerequisites
- Completed all previous notebooks (1.1-1.7)
- Understanding of experiment tracking and tracing

### Estimated Time: 25-30 minutes

---
## Step 1: System Architecture

### RAG Pipeline Flow

```
User Query
    ↓
┌─────────────────────────────────────┐
│  Query Processing (traced)          │
│  - Validation                       │
│  - Preprocessing                    │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│  Embedding Generation (traced)      │
│  - OpenAI embeddings                │
│  - Caching                          │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│  Vector Search (traced)             │
│  - Cosine similarity                │
│  - Top-k retrieval                  │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│  Context Assembly (traced)          │
│  - Reranking (optional)             │
│  - Prompt construction              │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│  LLM Generation (auto-traced)       │
│  - GPT-4o-mini                      │
│  - Streaming (optional)             │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│  Response Validation (traced)       │
│  - Quality checks                   │
│  - Formatting                       │
└─────────────────────────────────────┘
    ↓
Final Answer + Metadata
```

### What Gets Tracked

**Experiment Level:**
- Model configuration
- Prompt versions
- Hyperparameters (top_k, temperature)

**Run Level:**
- Individual queries
- Performance metrics
- Cost per query

**Trace Level:**
- Every operation's timing
- Inputs and outputs
- Custom attributes

---
## Step 2: Environment Setup

In [ ]:
import os
import time
import json
import hashlib
from typing import List, Dict, Optional
from datetime import datetime

import numpy as np
import mlflow
from openai import OpenAI
from dotenv import load_dotenv

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("./mlruns")
mlflow.set_experiment("10-complete-rag-system")

# Enable autologging
mlflow.openai.autolog()

# Initialize OpenAI
client = OpenAI()

print("✅ Environment configured for production RAG system")
print(f"   MLflow tracking: {mlflow.get_tracking_uri()}")
print(f"   Experiment: {mlflow.get_experiment_by_name('10-complete-rag-system').name}")

---
## Step 3: Document Store and Embeddings

In production, you'd use a vector database like Pinecone, Weaviate, or ChromaDB.
For this tutorial, we'll use in-memory storage.

In [ ]:
# Sample document corpus (in production, load from database)
DOCUMENT_STORE = {
    "doc1": "MLflow is an open source platform for the complete machine learning lifecycle. It provides experiment tracking, model packaging, and deployment capabilities across various ML frameworks.",
    "doc2": "MLflow Tracing provides comprehensive observability for GenAI applications. It captures LLM calls, retrieval steps, tool usage, and agent reasoning with full input/output visibility.",
    "doc3": "MLflow integrates with 30+ frameworks including OpenAI, Anthropic, LangChain, LlamaIndex, DSPy, and AutoGen. Each integration provides automatic tracing and experiment tracking.",
    "doc4": "MLflow Evaluation enables systematic testing of GenAI applications using LLM-as-judge metrics, custom scorers, and human feedback. It supports both batch and online evaluation.",
    "doc5": "MLflow Prompt Registry allows teams to version, share, and manage prompts centrally. It tracks which prompts are used in which experiments and enables A/B testing.",
    "doc6": "MLflow supports collaborative development with experiment sharing, model versioning, and deployment tracking. Teams can compare results and iterate systematically.",
    "doc7": "MLflow provides cost tracking for LLM applications by monitoring token usage, API calls, and compute resources. This helps teams optimize spending and budget effectively.",
    "doc8": "MLflow is fully open source and vendor-neutral, ensuring no lock-in. It works with any cloud provider, ML framework, or LLM provider.",
}

print(f"📚 Document store initialized with {len(DOCUMENT_STORE)} documents")

In [ ]:
# Cache for embeddings (in production, use Redis or similar)
EMBEDDING_CACHE = {}

@mlflow.trace(name="embed_text", span_type="EMBEDDING")
def embed_text(text: str) -> List[float]:
    """
    Generate embeddings with caching.
    """
    # Check cache
    cache_key = hashlib.md5(text.encode()).hexdigest()
    
    if cache_key in EMBEDDING_CACHE:
        mlflow.log_span_attribute("cache_hit", True)
        return EMBEDDING_CACHE[cache_key]
    
    mlflow.log_span_attribute("cache_hit", False)
    mlflow.log_span_attribute("text_length", len(text))
    
    # Generate embedding
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    
    embedding = response.data[0].embedding
    mlflow.log_span_attribute("embedding_dim", len(embedding))
    
    # Cache for future use
    EMBEDDING_CACHE[cache_key] = embedding
    
    return embedding

print("✅ Embedding function defined with caching")

In [ ]:
# Pre-compute document embeddings
print("\n🔄 Computing document embeddings...")

DOC_EMBEDDINGS = {}

for doc_id, text in DOCUMENT_STORE.items():
    DOC_EMBEDDINGS[doc_id] = embed_text(text)
    print(f"  ✓ {doc_id}")

print(f"\n✅ {len(DOC_EMBEDDINGS)} documents embedded and cached")

---
## Step 4: Query Processing and Validation

In [ ]:
@mlflow.trace(name="validate_query", span_type="PARSER")
def validate_query(query: str) -> Dict:
    """
    Validate and preprocess user query.
    """
    mlflow.log_span_attribute("original_length", len(query))
    
    # Basic validation
    if not query or len(query.strip()) == 0:
        mlflow.log_span_attribute("validation_error", "empty_query")
        raise ValueError("Query cannot be empty")
    
    if len(query) > 500:
        mlflow.log_span_attribute("validation_warning", "query_too_long")
        query = query[:500]  # Truncate
    
    # Preprocess
    processed_query = query.strip()
    
    mlflow.log_span_attribute("processed_length", len(processed_query))
    mlflow.log_span_attribute("validation_passed", True)
    
    return {
        "original": query,
        "processed": processed_query,
        "valid": True
    }

print("✅ Query validation function defined")

---
## Step 5: Semantic Search with Similarity Scoring

In [ ]:
@mlflow.trace(name="semantic_search", span_type="RETRIEVER")
def search_documents(
    query_embedding: List[float],
    doc_embeddings: Dict[str, List[float]],
    top_k: int = 3,
    min_score: float = 0.7
) -> List[Dict]:
    """
    Search for most relevant documents using cosine similarity.
    """
    mlflow.log_span_attribute("corpus_size", len(doc_embeddings))
    mlflow.log_span_attribute("top_k", top_k)
    mlflow.log_span_attribute("min_score", min_score)
    
    # Calculate cosine similarity for all documents
    scores = {}
    for doc_id, doc_emb in doc_embeddings.items():
        similarity = np.dot(query_embedding, doc_emb) / \
                     (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        scores[doc_id] = float(similarity)
    
    # Sort by score and filter by minimum threshold
    sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    filtered_docs = [(doc_id, score) for doc_id, score in sorted_docs if score >= min_score]
    
    # Get top k
    top_docs = filtered_docs[:top_k]
    
    results = [
        {
            "doc_id": doc_id,
            "score": score,
            "text": DOCUMENT_STORE[doc_id]
        }
        for doc_id, score in top_docs
    ]
    
    # Log metrics
    if results:
        mlflow.log_span_attribute("num_results", len(results))
        mlflow.log_span_attribute("top_score", results[0]["score"])
        mlflow.log_span_attribute("avg_score", np.mean([r["score"] for r in results]))
        mlflow.log_span_attribute("min_result_score", results[-1]["score"])
    else:
        mlflow.log_span_attribute("num_results", 0)
        mlflow.log_span_attribute("retrieval_warning", "no_docs_above_threshold")
    
    return results

print("✅ Semantic search function defined")

---
## Step 6: Context Assembly and Prompt Construction

In [ ]:
@mlflow.trace(name="assemble_context", span_type="PARSER")
def assemble_context(query: str, retrieved_docs: List[Dict]) -> str:
    """
    Assemble context from retrieved documents and construct prompt.
    """
    mlflow.log_span_attribute("num_docs", len(retrieved_docs))
    
    if not retrieved_docs:
        mlflow.log_span_attribute("context_warning", "no_docs_retrieved")
        return None
    
    # Format context
    context_parts = []
    for i, doc in enumerate(retrieved_docs, 1):
        context_parts.append(
            f"[Document {i}] (Relevance: {doc['score']:.2f})\n{doc['text']}"
        )
    
    context = "\n\n".join(context_parts)
    
    # Construct prompt with system instructions
    prompt = f"""You are a helpful AI assistant that answers questions based on provided context.

INSTRUCTIONS:
- Answer the question using ONLY the information in the context below
- If the answer is not in the context, say "I don't have enough information to answer that"
- Be concise but complete
- Cite which document(s) you used if relevant

CONTEXT:
{context}

QUESTION: {query}

ANSWER:"""
    
    mlflow.log_span_attribute("context_length", len(context))
    mlflow.log_span_attribute("prompt_length", len(prompt))
    
    return prompt

print("✅ Context assembly function defined")

---
## Step 7: LLM Response Generation

In [ ]:
@mlflow.trace(name="generate_response", span_type="LLM")
def generate_answer(
    prompt: str,
    model: str = "gpt-4o-mini",
    temperature: float = 0.1,
    max_tokens: int = 300
) -> Dict:
    """
    Generate answer using LLM.
    """
    mlflow.log_span_attribute("model", model)
    mlflow.log_span_attribute("temperature", temperature)
    mlflow.log_span_attribute("max_tokens", max_tokens)
    
    if not prompt:
        mlflow.log_span_attribute("generation_error", "empty_prompt")
        raise ValueError("Prompt cannot be empty")
    
    # Call LLM (automatically traced by OpenAI autolog)
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    answer = response.choices[0].message.content
    
    # Log generation metrics
    mlflow.log_span_attribute("answer_length", len(answer))
    mlflow.log_span_attribute("prompt_tokens", response.usage.prompt_tokens)
    mlflow.log_span_attribute("completion_tokens", response.usage.completion_tokens)
    mlflow.log_span_attribute("total_tokens", response.usage.total_tokens)
    mlflow.log_span_attribute("finish_reason", response.choices[0].finish_reason)
    
    return {
        "answer": answer,
        "tokens": response.usage.total_tokens,
        "model": model,
        "finish_reason": response.choices[0].finish_reason
    }

print("✅ Answer generation function defined")

---
## Step 8: Response Validation and Post-Processing

In [ ]:
@mlflow.trace(name="validate_response", span_type="PARSER")
def validate_response(answer: str, min_length: int = 10) -> Dict:
    """
    Validate and post-process generated response.
    """
    mlflow.log_span_attribute("min_length_threshold", min_length)
    mlflow.log_span_attribute("answer_length", len(answer))
    
    issues = []
    
    # Check length
    if len(answer) < min_length:
        issues.append("too_short")
    
    # Check for common error patterns
    error_patterns = [
        "I don't have enough information",
        "I cannot answer",
        "I apologize"
    ]
    
    for pattern in error_patterns:
        if pattern.lower() in answer.lower():
            issues.append(f"contains_{pattern.replace(' ', '_').lower()}")
    
    mlflow.log_span_attribute("validation_issues", ",".join(issues) if issues else "none")
    mlflow.log_span_attribute("is_valid", len(issues) == 0)
    
    return {
        "is_valid": len(issues) == 0,
        "issues": issues,
        "answer": answer
    }

print("✅ Response validation function defined")

---
## Step 9: Complete RAG Pipeline

In [ ]:
@mlflow.trace(name="rag_pipeline", span_type="CHAIN")
def rag_qa_system(
    user_query: str,
    top_k: int = 3,
    min_score: float = 0.7,
    model: str = "gpt-4o-mini",
    temperature: float = 0.1
) -> Dict:
    """
    Complete RAG pipeline with full observability.
    
    Returns:
        Dict with answer, metadata, and status
    """
    # Start MLflow run for tracking
    with mlflow.start_run(run_name="rag_query"):
        
        # Log parameters
        mlflow.log_params({
            "query": user_query[:100],  # Truncate for readability
            "top_k": top_k,
            "min_score": min_score,
            "model": model,
            "temperature": temperature,
            "timestamp": datetime.now().isoformat()
        })
        
        try:
            # Step 1: Validate query
            validation_result = validate_query(user_query)
            processed_query = validation_result["processed"]
            
            # Step 2: Generate query embedding
            query_embedding = embed_text(processed_query)
            
            # Step 3: Search documents
            retrieved_docs = search_documents(
                query_embedding,
                DOC_EMBEDDINGS,
                top_k=top_k,
                min_score=min_score
            )
            
            if not retrieved_docs:
                mlflow.log_metric("retrieval_success", 0)
                return {
                    "status": "no_relevant_docs",
                    "answer": "I couldn't find relevant information to answer your question.",
                    "query": user_query,
                    "retrieved_docs": []
                }
            
            mlflow.log_metric("num_docs_retrieved", len(retrieved_docs))
            mlflow.log_metric("avg_relevance_score", 
                            np.mean([d["score"] for d in retrieved_docs]))
            mlflow.log_metric("retrieval_success", 1)
            
            # Step 4: Assemble context and construct prompt
            prompt = assemble_context(processed_query, retrieved_docs)
            
            # Step 5: Generate answer
            generation_result = generate_answer(
                prompt,
                model=model,
                temperature=temperature
            )
            
            # Step 6: Validate response
            validation = validate_response(generation_result["answer"])
            
            # Log final metrics
            mlflow.log_metric("total_tokens", generation_result["tokens"])
            mlflow.log_metric("answer_valid", 1 if validation["is_valid"] else 0)
            mlflow.log_metric("answer_length", len(generation_result["answer"]))
            
            # Log artifacts
            mlflow.log_text(user_query, "query.txt")
            mlflow.log_text(generation_result["answer"], "answer.txt")
            mlflow.log_text(prompt, "full_prompt.txt")
            mlflow.log_dict(
                {"docs": retrieved_docs},
                "retrieved_docs.json"
            )
            
            # Construct result
            result = {
                "status": "success",
                "query": user_query,
                "answer": generation_result["answer"],
                "retrieved_docs": retrieved_docs,
                "metadata": {
                    "num_docs": len(retrieved_docs),
                    "avg_relevance": float(np.mean([d["score"] for d in retrieved_docs])),
                    "tokens_used": generation_result["tokens"],
                    "model": model,
                    "is_valid": validation["is_valid"],
                    "validation_issues": validation["issues"]
                }
            }
            
            return result
            
        except Exception as e:
            # Log error
            mlflow.log_param("error_type", type(e).__name__)
            mlflow.log_param("error_message", str(e))
            mlflow.log_metric("pipeline_success", 0)
            
            return {
                "status": "error",
                "query": user_query,
                "error": str(e),
                "error_type": type(e).__name__
            }

print("✅ Complete RAG pipeline defined")

---
## Step 10: Testing the RAG System

In [ ]:
# Test queries
test_queries = [
    "What tracing capabilities does MLflow provide?",
    "How does MLflow help with cost tracking?",
    "Can MLflow integrate with LangChain?",
    "What is the purpose of MLflow Prompt Registry?",
]

print("\n🧪 Testing RAG System\n")
print("="*80)

results = []

for i, query in enumerate(test_queries, 1):
    print(f"\nQuery {i}: {query}")
    print("-"*80)
    
    start_time = time.time()
    result = rag_qa_system(query, top_k=3, min_score=0.7)
    latency = time.time() - start_time
    
    if result["status"] == "success":
        print(f"\nAnswer: {result['answer']}")
        print(f"\nMetadata:")
        print(f"  - Documents used: {result['metadata']['num_docs']}")
        print(f"  - Avg relevance: {result['metadata']['avg_relevance']:.3f}")
        print(f"  - Tokens: {result['metadata']['tokens_used']}")
        print(f"  - Latency: {latency:.2f}s")
        print(f"  - Valid: {result['metadata']['is_valid']}")
        
        results.append({
            "query": query,
            "success": True,
            "latency": latency,
            "tokens": result['metadata']['tokens_used']
        })
    else:
        print(f"\nError: {result.get('error', 'Unknown error')}")
        results.append({
            "query": query,
            "success": False,
            "latency": latency
        })

print("\n" + "="*80)
print("\n✅ All queries processed!")

---
## Step 11: Performance Analysis

In [ ]:
# Analyze performance
print("\n📊 Performance Summary\n")
print("="*60)

successful = [r for r in results if r["success"]]

if successful:
    latencies = [r["latency"] for r in successful]
    tokens = [r["tokens"] for r in successful]
    
    print(f"Success Rate: {len(successful)}/{len(results)} ({len(successful)/len(results)*100:.1f}%)")
    print(f"\nLatency Stats:")
    print(f"  Average: {np.mean(latencies):.2f}s")
    print(f"  Min: {np.min(latencies):.2f}s")
    print(f"  Max: {np.max(latencies):.2f}s")
    print(f"  Std Dev: {np.std(latencies):.2f}s")
    
    print(f"\nToken Usage:")
    print(f"  Average: {np.mean(tokens):.0f} tokens")
    print(f"  Total: {np.sum(tokens):.0f} tokens")
    print(f"  Est. Cost: ${np.sum(tokens) * 0.15 / 1_000_000:.6f}")
    
    print(f"\nCache Performance:")
    print(f"  Embedding cache hits: {len(EMBEDDING_CACHE)} embeddings cached")
    
print("\n" + "="*60)

---
## Step 12: RAG Evaluation with RAGAS Metrics

Now let's evaluate our RAG system using RAGAS (Retrieval-Augmented Generation Assessment) metrics. These metrics help assess the quality of both retrieval and generation.

| Metric | What It Measures |
|--------|-----------------|
| **Faithfulness** | Is the answer grounded in the retrieved context? |
| **Context Precision** | Are the retrieved documents relevant to the question? |
| **RetrievalGroundedness** | Does the answer stay true to retrieved information? |

In [ ]:
from mlflow.genai.scorers import RetrievalGroundedness
from mlflow.genai.scorers.ragas import Faithfulness, ContextPrecision

# Judge model for RAGAS evaluation
JUDGE_MODEL = "openai:/gpt-4o-mini"

# Initialize RAGAS scorers
faithfulness_scorer = Faithfulness(model=JUDGE_MODEL)
context_precision_scorer = ContextPrecision(model=JUDGE_MODEL)
groundedness_scorer = RetrievalGroundedness(model=JUDGE_MODEL)

print("✅ RAGAS scorers initialized:")
print("   - Faithfulness")
print("   - ContextPrecision")
print("   - RetrievalGroundedness")

In [ ]:
# Prepare RAG evaluation dataset from our test results
# We need to format the data with inputs, outputs, and context

rag_eval_dataset = []

for query in test_queries:
    # Run the RAG pipeline to get results (uses cached embeddings)
    result = rag_qa_system(query, top_k=3, min_score=0.7)
    
    if result["status"] == "success":
        # Extract context from retrieved docs
        context = [doc["text"] for doc in result["retrieved_docs"]]
        
        rag_eval_dataset.append({
            "inputs": {"question": query},
            "outputs": {"response": result["answer"]},
            "context": {"retrieved_context": context}
        })

print(f"✅ RAG evaluation dataset prepared with {len(rag_eval_dataset)} examples")
for i, item in enumerate(rag_eval_dataset, 1):
    print(f"   {i}. {item['inputs']['question'][:50]}...")

In [ ]:
# Run RAGAS evaluation
print("🔄 Running RAGAS evaluation on RAG outputs...\n")

ragas_results = mlflow.genai.evaluate(
    data=rag_eval_dataset,
    scorers=[
        faithfulness_scorer,
        context_precision_scorer,
        groundedness_scorer
    ]
)

print("\n✅ RAGAS evaluation complete!")
print("\n📊 RAGAS Metrics Summary:")
print("-" * 50)
for metric_name, value in ragas_results.metrics.items():
    if isinstance(value, float):
        print(f"  {metric_name}: {value:.3f}")
    else:
        print(f"  {metric_name}: {value}")

---
## Step 13: Viewing Results in MLflow UI

Now let's explore what was tracked in MLflow.

---
## Step 12: Viewing Results in MLflow UI

Now let's explore what was tracked in MLflow.

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║         Analyzing Results in MLflow UI                       ║
╚══════════════════════════════════════════════════════════════╝

🔍 EXPERIMENTS VIEW:
   Navigate to: http://localhost:5000
   Select: "10-complete-rag-system" experiment
   
   You'll see:
   - All RAG query runs
   - Parameters (query, model, top_k)
   - Metrics (tokens, relevance, latency)
   - Artifacts (queries, answers, prompts)

📊 COMPARING RUNS:
   1. Select multiple runs
   2. Click "Compare"
   3. View side-by-side:
      - Which queries used most tokens?
      - Which had highest relevance scores?
      - Performance variations

🌳 TRACES VIEW:
   Click "Traces" tab to see:
   
   Timeline visualization:
   rag_pipeline (CHAIN) ━━━━━━━━━━━━━━━━━━━━ 2.5s
   ├─ validate_query (PARSER) ━━ 0.01s
   ├─ embed_text (EMBEDDING) ━━━━ 0.3s
   ├─ semantic_search (RETRIEVER) ━ 0.05s
   ├─ assemble_context (PARSER) ━ 0.02s
   ├─ generate_response (LLM) ━━━━━ 2.0s
   │  └─ OpenAI API call ━━━━━━━━━ 1.9s
   └─ validate_response (PARSER) ━ 0.01s

🔎 SPAN DETAILS:
   Click on any span to see:
   - Inputs and outputs
   - Custom attributes
   - Timing information
   - Cache hit status
   - Relevance scores

📈 KEY INSIGHTS:
   1. Performance Bottlenecks:
      - Which step takes longest?
      - Is it the LLM or retrieval?
   
   2. Quality Metrics:
      - Average relevance scores
      - Documents per query
      - Answer validation rates
   
   3. Cost Analysis:
      - Token usage per query
      - Cache effectiveness
      - Cost per operation
   
   4. Error Patterns:
      - Failed queries
      - Low relevance scores
      - Validation issues

💡 OPTIMIZATION OPPORTUNITIES:
   Based on traces, you can:
   - Adjust top_k if retrieval is slow
   - Increase min_score if quality is poor
   - Optimize prompts to reduce tokens
   - Add more aggressive caching
   - Implement parallel retrieval

""")

---
## Summary

Congratulations! You've built a complete, production-ready RAG system with:

### ✅ Core Functionality
- Document embedding and indexing
- Semantic search with cosine similarity
- Context-aware response generation
- Input validation and output validation

### ✅ Observability
- Complete experiment tracking
- End-to-end distributed tracing
- Custom span attributes
- Performance metrics
- Cost tracking

### ✅ Production Features
- Embedding caching
- Error handling
- Query validation
- Response validation
- Relevance scoring

### ✅ RAG Evaluation
- RAGAS metrics (Faithfulness, Context Precision)
- Retrieval groundedness scoring
- LLM-as-Judge evaluation patterns

### 🎯 What You Can Do Next

**Immediate Improvements:**
1. Add a vector database (Pinecone, Weaviate, ChromaDB)
2. Implement streaming responses
3. Add reranking for better relevance
4. Create a web UI (Streamlit, Gradio)
5. Set up monitoring dashboards

**Production Enhancements:**
1. Deploy as an API (FastAPI)
2. Add rate limiting
3. Implement user authentication
4. Set up A/B testing framework
5. Create alerting for quality/cost

### 📚 Complete Tutorial Series

You've now completed Tutorial 1:

1. ✅ **Setup and Introduction**
2. ✅ **Experiment Tracking for LLMs**
3. ✅ **Introduction to Tracing**
4. ✅ **Manual Tracing and Advanced Observability**
5. ✅ **Prompt Management**
6. ✅ **Framework Integrations**
7. ✅ **Evaluating Agents**
8. ✅ **Complete RAG Application** (This notebook!)

### 🚀 Next Tutorial Series

Ready to go deeper?

- **Tutorial 2**: Prompt Engineering and Version Control
- **Tutorial 3**: Advanced Tracing and Debugging
- **Tutorial 4**: Advanced Agent Evaluation
- **Tutorial 5**: Optimizing Prompts for Performance

### 🎉 Congratulations!

You now have:
- A production-ready RAG system
- Complete observability into your GenAI app
- Understanding of MLflow's core capabilities
- Patterns for building scalable LLM applications
- RAG evaluation using RAGAS metrics

**Keep building, keep learning, and happy coding!** 🚀